<a href="https://colab.research.google.com/github/AlexeyRogS/cv_course/blob/main/week4/TrickingCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import models, transforms

In [ ]:
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.efficientnet_b3(weights=models.EfficientNet_B3_Weights.DEFAULT)
model = model.to(device)
model.eval();

In [ ]:
t = transforms.Compose([
    transforms.Resize(320, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Donald_Trump_official_portrait.jpg/1200px-Donald_Trump_official_portrait.jpg -O "img.jpg"
!wget https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json -O "labels.json"

In [ ]:
import json
with open("labels.json", 'r') as f:
    labels = json.loads(f.read())
labels = np.array(labels)
labels[:10]

In [ ]:
img = cv2.imread("img.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
N = 5
x = t(Image.open("img.jpg"))[None,...].to(device)
# x = models.EfficientNet_B3_Weights.IMAGENET1K_V1.transforms()(Image.open("img.jpg"))[None,...].to(device)
with torch.no_grad():
    pred = model(x).cpu().numpy()[0]
    pred_ids = np.argsort(pred)[-N:][::-1]
labels[pred_ids]

In [ ]:
from torch.optim import Adam
import torch.nn as nn
gt = torch.tensor([97], dtype=torch.long, device=device)
x = t(Image.open("img.jpg"))[None,...].to(device)
x.requires_grad = True
opt = Adam([x], lr=0.001)
criterion = nn.CrossEntropyLoss()
loss_hist = []
for _ in range(100):
    pred = model(x)
    loss = criterion(pred, gt)
    opt.zero_grad()
    loss.backward()
    opt.step()
    loss_hist.append(loss.item())

In [ ]:
plt.plot(loss_hist)
plt.show()

In [ ]:
with torch.no_grad():
    pred = model(x).cpu().numpy()[0]
    pred_ids = np.argsort(pred)[-N:][::-1]
labels[pred_ids]

In [ ]:
res_img = np.transpose(x[0].detach().cpu().numpy(), axes=(1, 2, 0)) * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
res_img = np.clip(res_img * 255, 0, 255).astype(np.uint8)
plt.imshow(res_img)
plt.show()